In [ ]:
import pandas as pd

synonyms_file = r'C:\Users\Nattapot\Desktop\thesis_ef\extract_data\synonyms.txt'
with open(synonyms_file, encoding="utf-8") as f:
    synonyms_list = [line.strip().split(", ") for line in f.readlines()]

dataset_file = r'C:\Users\Nattapot\Desktop\thesis_ef\evaluate\emission_factor_20250210.csv'
df = pd.read_csv(dataset_file, header=None) 
df['combined'] = df[[0, 1, 2]].apply(lambda row: " ".join(str(value) for value in row if pd.notnull(value)), axis=1)

results = []
for synonyms in synonyms_list:
    keyword = synonyms[0] 
    matched_rows = []
    for index, row in df.iterrows():
        row_text = row['combined']
        if any(word in row_text for word in synonyms):
            matched_rows.append(index) 

    results.append({"Keyword": keyword, "Synonyms": ", ".join(synonyms), "Rows Found": matched_rows})
result_df = pd.DataFrame(results)

result_df.to_csv("ground_truth.csv",index=False,encoding='utf-8-sig')

In [ ]:
from elasticsearch import Elasticsearch


es = Elasticsearch("https://localhost:9200",
                   basic_auth=("elastic","JODDaUKomoKuPHFM2zEc"),
                   ca_certs="C:/Users/Nattapot/Documents/elasticsearch-8.17.0/config/certs/http_ca.crt"
)


def search_and_evaluate(query, ground_truth, index="emission_data_upsert", size=10):
    """
    ค้นหาใน Elasticsearch และคำนวณ Precision และ Recall
    
    Parameters:
    - query (str): คำค้นหาที่ใช้ใน Elasticsearch
    - ground_truth (dict): Ground Truth ที่ระบุเอกสารที่เกี่ยวข้องสำหรับคำค้น
    - index (str): ชื่อ Index ใน Elasticsearch
    - size (int): จำนวนผลลัพธ์สูงสุดที่คืน
    
    Returns:
    - dict: ผลลัพธ์ที่รวม Precision, Recall, และรายละเอียดอื่น ๆ
    """
    try:
        # ตรวจสอบว่ามี Query หรือไม่
        if not query:
            return {"error": "No query provided."}

        # ค้นหาใน Elasticsearch
        response = es.search(index=index, body={
            "query": {
                "multi_match": {
                    "query": query,
                    "fields": ["ชื่อ^3", "รายละเอียด", "กลุ่ม"],
                    "type": "best_fields",
                    "operator": "and"
                }
            },
            "size": size
        })

        # ดึง _id ของผลลัพธ์
        retrieved_ids = [hit["_id"] for hit in response['hits']['hits']]
        
        # เปรียบเทียบกับ Ground Truth
        relevant_ids = ground_truth.get(query, [])  # Ground Truth ที่เกี่ยวข้องกับ Query นี้
        tp = len(set(retrieved_ids) & set(relevant_ids))  # True Positives
        fp = len(set(retrieved_ids) - set(relevant_ids))  # False Positives
        fn = len(set(relevant_ids) - set(retrieved_ids))  # False Negatives

        # คำนวณ Precision และ Recall
        precision = tp / (tp + fp) if (tp + fp) > 0 else 0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0

        # คืนผลลัพธ์
        return {
            "query": query,
            "precision": precision,
            "recall": recall,
            "retrieved_ids": retrieved_ids,
            "relevant_ids": relevant_ids,
            "true_positives": tp,
            "false_positives": fp,
            "false_negatives": fn,
        }
    
    except Exception as e:
        return {"error": str(e)}

In [71]:
gt = r'C:\Users\Nattapot\Desktop\thesis_ef\evaluate\ground_truth.csv'
df = pd.read_csv(gt) 
synonym_dict = {row['Keyword']:row['Synonyms'].replace(' ','').split(',') for _,row in df.iterrows()}
ground_truth = {row['Keyword']:row['Rows Found'] for _,row in df.iterrows()}

synonym_dict
# synonym_dict = {
#     "Anthracite": ["Anthracite", "แอนทราไซต์", "ถ่านหินชนิดแข็ง", "ถ่านหินคุณภาพสูง"],
#     "Bagasse": ["Bagasse", "ชานอ้อย", "กากอ้อย"],
#     "Benzene": ["Benzene", "เบนซีน"]
# }


# ground_truth = {
#     "Anthracite": ["1", "2", "3"],
#     "Bagasse": ["4", "5", "6"],
#     "Benzene": ["7", "8"]
# }


{'Agriculture': ['Agriculture',
  'agricultural',
  'farming',
  'เกษตร',
  'เกษตรกรรม',
  'กสิกรรม',
  'การเพาะปลูก',
  'การทำไร่ทำนา',
  'การทำการเกษตร'],
 'Anthracite': ['Anthracite',
  'แอนทราไซต์',
  'ถ่านหินชนิดแข็ง',
  'ถ่านหินคุณภาพสูง'],
 'Bagasse': ['Bagasse', 'ชานอ้อย', 'กากอ้อย'],
 'Benzene': ['Benzene', 'เบนซีน'],
 'Biogas': ['Biogas', 'ไบโอแก๊ส', 'ก๊าซชีวภาพ', 'แก๊สชีวภาพ'],
 'Caprolactam': ['Caprolactam', 'คาโพรแลคแทม', 'แคปโรแลคแตม'],
 'Carbon dioxide': ['Carbondioxide',
  'CO2',
  'คาร์บอนไดออกไซด์',
  'ก๊าซคาร์บอนไดออกไซด์'],
 'Cob': ['Cob', 'ฝักข้าวโพด', 'แกนข้าวโพด', 'ซังข้าวโพด'],
 'Compressed Natural Gas': ['CompressedNaturalGas',
  'CNG',
  'NGV',
  'ก๊าซธรรมชาติอัด',
  'ก๊าซเอ็นจีวี',
  'ก๊าซธรรมชาติสำหรับยานยนต์'],
 'Cyclohexane': ['Cyclohexane', 'ไซโคลเฮกเซน'],
 'Diesel': ['Diesel', 'Dieseloil', 'น้ำมันดีเซล', 'ดีเซล'],
 'Ethane': ['Ethane', 'อีเทน', 'ก๊าซอีเทน'],
 'Ethylene': ['Ethylene', 'C2H4', 'เอทิลีน', 'ก๊าซเอทิลีน'],
 'FUEL WOOD': ['FUELWOOD', 'ฟืน', 'เ

In [80]:
from elasticsearch import Elasticsearch
import pandas as pd

es = Elasticsearch("https://localhost:9200",
                   basic_auth=("elastic","JODDaUKomoKuPHFM2zEc"),
                   ca_certs="C:/Users/Nattapot/Documents/elasticsearch-8.17.0/config/certs/http_ca.crt"
)


def search_and_evaluate_synonym(query, synonym_dict, ground_truth, index="emission_data_upsert", size=10):
    """
    ค้นหาใน Elasticsearch พร้อมใช้ Synonym Matching และคำนวณ Precision และ Recall
    """
    try:
        if not query:
            return {"error": "No query provided."}

        # ขยาย Query ด้วย Synonym Dictionary
        expanded_queries = synonym_dict.get(query, [query])  # ใช้คำเดิมถ้าไม่มีใน Dictionary
        print(expanded_queries)
        # สร้าง Query Elasticsearch
        response = es.search(index=index, body={
            "query": {
                "multi_match": {
                    "query": " ".join(expanded_queries),  # ใช้ Synonyms ทั้งหมดรวมกัน
                    "fields": ["ชื่อ^3", "รายละเอียด", "กลุ่ม"],
                    "type": "best_fields",
                    "operator": "or"  # ใช้ "or" เพื่อให้ตรงกับคำใดคำหนึ่งใน Synonym
                }
            },
            "size": size
        })


        retrieved_ids = [hit["_id"] for hit in response['hits']['hits']]
        
        relevant_ids = ground_truth.get(query, [])
        tp = len(set(retrieved_ids) & set(relevant_ids))  # True Positives
        fp = len(set(retrieved_ids) - set(relevant_ids))  # False Positives
        fn = len(set(relevant_ids) - set(retrieved_ids))  # False Negatives

        # คำนวณ Precision และ Recall
        precision = tp / (tp + fp) if (tp + fp) > 0 else 0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0

        return {
            "query": query,
            "expanded_queries": expanded_queries,
            "precision": precision,
            "recall": recall,
            "retrieved_ids": retrieved_ids,
            "relevant_ids": relevant_ids,
            "true_positives": tp,
            "false_positives": fp,
            "false_negatives": fn,
        }
    
    except Exception as e:
        return {"error": str(e)}


In [81]:
result = search_and_evaluate_synonym("Anthracite", synonym_dict, ground_truth)
result

['Anthracite', 'แอนทราไซต์', 'ถ่านหินชนิดแข็ง', 'ถ่านหินคุณภาพสูง']


{'query': 'Anthracite',
 'expanded_queries': ['Anthracite',
  'แอนทราไซต์',
  'ถ่านหินชนิดแข็ง',
  'ถ่านหินคุณภาพสูง'],
 'precision': 0.0,
 'recall': 0.0,
 'retrieved_ids': ['601'],
 'relevant_ids': '[601]',
 'true_positives': 0,
 'false_positives': 1,
 'false_negatives': 5}